<a href="https://colab.research.google.com/github/DmitryKutsev/eng_to_jap_translator/blob/main/my_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Neural Machine Translation.**

Проект - модель генерации перевода с английского на японский. В основе лежит модель seq2seq, но если получится, то я попробую немного усложнить в дальнейшем.

https://arxiv.org/pdf/1706.08198.pdf 
https://www.aclweb.org/anthology/W14-7008.pdf - статьи, на которые примерно ориентировался.

В качестве токенизатора японского языка использовал tinysegmenter: https://pypi.org/project/tinysegmenter/.

В качестве корпусов:

Сначала корпус Kurohashi-Kawahara Lab:  http://nlp.ist.i.kyoto-u.ac.jp/EN/?JEC%20Basic%20Sentence%20Data


Потом нашел корпус побольше, параллельный корпус англо-японских субтитров https://nlp.stanford.edu/projects/jesc/data/raw.tar.gz


In [1]:
!pip install tinysegmenter

  Created wheel for tinysegmenter: filename=tinysegmenter-0.4-cp36-none-any.whl size=13537 sha256=f2ebe8de3daffd46f4d8b46ecf3144ac458bcc71bf51e00f44b5b231aeef7030
  Stored in directory: /root/.cache/pip/wheels/68/71/2b/6402196bf28012826e507ef7b99df6ebd98cce78bd99023471
Successfully built tinysegmenter


In [2]:
import sys
import os
import math
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np

import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
import random
import spacy
import tinysegmenter

import torch
import torch.nn as nn
import random


In [4]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
spacy_en = spacy.load('en')

In [6]:
segmenter = tinysegmenter.TinySegmenter()

In [7]:
! wget https://nlp.stanford.edu/projects/jesc/data/raw.tar.gz

--2020-12-20 08:16:02--  https://nlp.stanford.edu/projects/jesc/data/raw.tar.gz
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102198198 (97M) [application/x-gzip]
Saving to: ‘raw.tar.gz’

raw.tar.gz          100%[===================>]  97.46M  28.1MB/s    in 5.7s    

2020-12-20 08:16:08 (17.0 MB/s) - ‘raw.tar.gz’ saved [102198198/102198198]



In [8]:
!tar -xzf raw.tar.gz

In [9]:
my_frame = pd.read_csv('raw/raw', sep='\t')

In [10]:

my_frame.columns = ['en', 'jp']
my_frame = my_frame[:400000]
my_frame = my_frame[my_frame.columns[::-1]]

In [11]:
my_frame

,jp,en
0,俺の相手は シャークだ。,my opponent is shark.
1,引き換えだ ある事とある物の,this is one thing in exchange for another.
2,もういいよ ごちそうさま ううん,"yeah, i'm fine."
3,もう会社には来ないでくれ 電話もするな,don't come to the office anymore. don't call m...
4,きれいだ。,looks beautiful.
...,...,...
399995,電話がインターネット接続しようとしてる,my phone is dialing for an internet connection.
399996,手札から アブサード・スティーラーを召喚!,i summon absurd stealer from my hand!
399997,いいえ その学位については,"yes, about your double degree."
399998,似てるって思うんだけどね。,i think...


In [12]:
segmenter.tokenize(my_frame['jp'][1])

['引き換え', 'だ', ' ', 'ある', '事', 'と', 'ある', '物', 'の']

In [13]:
[tok.text for tok in spacy_en.tokenizer(my_frame['en'][1])]

['this', 'is', 'one', 'thing', 'in', 'exchange', 'for', 'another', '.']

In [14]:
my_frame.to_csv('my_frame.csv', index=False)  

В какой-то момент я случайно убрал MAX_LEN, о чем забыл потом, и некоторое время не мог понять, почему на полных данных модель падает.

In [15]:
# MAX_LEN = 25
MAX_LEN = 20

def tokenize_jp(text):
    """
    Tokenizes JP text from a string into a list of strings
    """
    return segmenter.tokenize(text)[:MAX_LEN]


def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    res = [tok.text for tok in spacy_en.tokenizer(text)]
    return res[:MAX_LEN]

В качестве обработчика данных решил использовать инструменты torchtext.


In [16]:
SRC = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>')
TRG = Field(tokenize=tokenize_jp, init_token='<sos>', eos_token='<eos>')

In [17]:
dataset = TabularDataset(path='my_frame.csv', 
                         format='csv', 
                         fields=[ ('jp', TRG), ('en', SRC),],
                         skip_header=True)

In [18]:
train_data, valid_data, test_data = dataset.split(split_ratio=[0.7, 0.1, 0.2], 
                                            random_state=random.getstate())

In [71]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [20]:
print(vars(train_data.examples[0])['en'])

['for', 'the', 'countless', ',', 'countless', 'hours']


In [21]:
print (len(SRC.vocab), len(TRG.vocab))
print (SRC.vocab.freqs.most_common(10))
print (TRG.vocab.freqs.most_common(10))

14188 23113
[('.', 181545), (',', 95628), ('you', 70645), ('the', 70065), ('i', 66790), ('?', 52947), ('to', 49348), ('a', 41064), ('it', 39733), ("'s", 39706)]
[(' ', 199404), ('の', 107014), ('は', 80117), ('に', 71958), ('て', 66374), ('を', 61525), ('が', 57002), ('た', 56237), ('?', 41254), ('。', 37468)]


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 256

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_key=lambda x: len(x.jp), 
     sort_within_batch=False,
     device = device)


In [24]:

# for b in valid_iterator:
#     print (b.jp, b.en)
#     sys.exit()


In [25]:
print(vars(valid_iterator))

{'batch_size': 256, 'train': False, 'dataset': <torchtext.data.dataset.Dataset object at 0x7fe2b0c85f28>, 'batch_size_fn': None, 'iterations': 0, 'repeat': False, 'shuffle': False, 'sort': True, 'sort_within_batch': False, 'sort_key': <function <lambda> at 0x7fe2d4aa60d0>, 'device': device(type='cpu'), 'random_shuffler': <torchtext.data.utils.RandomShuffler object at 0x7fe2b0603860>, '_iterations_this_epoch': 0, '_random_state_this_epoch': None, '_restored_from_state': False}


In [26]:
print(len(valid_iterator))

313


Раздел с самими, собственно, сетями.

In [27]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))        
        outputs, (hidden, cell) = self.rnn(embedded)

        
        return hidden, cell

In [28]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):

            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        
        return outputs

In [30]:
len(SRC.vocab), len(TRG.vocab)

(14188, 23113)

In [31]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.6
DEC_DROPOUT = 0.6

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [32]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14188, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (dropout): Dropout(p=0.6, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(23113, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (fc_out): Linear(in_features=512, out_features=23113, bias=True)
    (dropout): Dropout(p=0.6, inplace=False)
  )
)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 28,762,441 trainable parameters


In [34]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
pad_idx = TRG.vocab.stoi['<pad>']
print(TRG.pad_token)  # <pad>
print(TRG.vocab.stoi[TRG.pad_token]) # 1 

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

<pad>
1


Обучение и оценка.

Сначала я поставил progress_bar, и вроде избавился от всех его глюков в колабе, но но в какой-то момент он опять начал странно работать, и за день до дедлайна я решил, что пока ограничусь выводом результатов в принт, а progress_bar вернется, если останется время его опять чинить.

In [36]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    my_losses = []
    for i, batch in enumerate(iterator):
        # progress_bar = tqdm(total=len(iterator), desc=f'{ i }')
        src = batch.en
        trg = batch.jp
        
        optimizer.zero_grad()
        
        output = model(src, trg)

        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        my_losses.append(loss.item())
        if i%10 == 0:
          print(f'fmean losses: { np.mean(my_losses[-1000:]) } ', f'iter { i }' )
        # progress_bar.set_postfix(loss=np.mean(my_losses[-1000:]),
                            # perplexity=np.exp(np.mean(my_losses[-1000:])))
        # progress_bar.update()
     # progress_bar.close()
    return epoch_loss / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.en
            trg = batch.jp

            output = model(src, trg, 0) 
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [38]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
import time

In [41]:

# for instance in list(tqdm._instances):
#   tqdm._decr_instances(instance)


In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Это проход уже по полным данным, что у меня некоторое время вообще не получалось.


In [43]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


fmean losses: 10.038658142089844  iter 0
fmean losses: 7.954061074690386  iter 10
fmean losses: 7.035599981035505  iter 20
fmean losses: 6.681584742761427  iter 30
fmean losses: 6.484833356810779  iter 40
fmean losses: 6.355862037808287  iter 50
fmean losses: 6.268964658018018  iter 60
fmean losses: 6.197297022376262  iter 70
fmean losses: 6.146051548145436  iter 80
fmean losses: 6.1046259481828296  iter 90
fmean losses: 6.0671673104314525  iter 100
fmean losses: 6.033696432371397  iter 110
fmean losses: 6.005044014985896  iter 120
fmean losses: 5.979149829340345  iter 130
fmean losses: 5.953744878160193  iter 140
fmean losses: 5.933502566735476  iter 150
fmean losses: 5.916575526598818  iter 160
fmean losses: 5.8988558367679  iter 170
fmean losses: 5.885104582454618  iter 180
fmean losses: 5.869276341343426  iter 190
fmean losses: 5.855923384576294  iter 200
fmean losses: 5.84154014677798  iter 210
fmean losses: 5.829636603998383  iter 220
fmean losses: 5.817247064598711  iter 230
fme

In [44]:
def translate_sentence(sentence,src_field,trg_field,model,device, max_len=50):
    model.eval()

    if isinstance(sentence,str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    #
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:-1] # remove <sos> and <eos>

In [45]:
example_idx = 24

src = vars(train_data.examples[example_idx])['en']
trg = vars(train_data.examples[example_idx])['jp']

print(f'src = {src}')
print(f'trg = {trg}')

translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

src = ['you', 'always', 'had', 'a', 'reason', '?']
trg = ['あん', 'た', 'は', 'いつも', '理由', 'が', 'あっ', 'た', 'の', '?']
predicted trg = ['<unk>', 'は', ' ', 'の', '?']


По визуальной оценке результат просто ужасный. Видимо, я слишком высоко поднял фильтрацию повторяющихся слов(сейчас ве, что меньше 4х - фильтруется.)


In [47]:
model_save_name = 'tut1-model.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [48]:
translation2 = translate_sentence('I go to school', SRC, TRG, model, device)

print(f'predicted trg = {translation2}')

predicted trg = ['<unk>', 'の', '<unk>']


In [49]:
translation2 = translate_sentence('I want to eat and drink', SRC, TRG, model, device)

print(f'predicted trg = {translation2}')

predicted trg = ['<unk>', 'の', ' ', 'に', ' ']


In [50]:
translation2 = translate_sentence('I hate school and study', SRC, TRG, model, device)

print(f'predicted trg = {translation2}')

predicted trg = ['<unk>', 'の', ' ', 'の', ' ']


В качестве оценки использовал метрику bleu из библиотеки NLTK


In [66]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate import bleu_score

In [64]:
for i in train_data.examples[:3]:
  print(vars(i)['jp'])
  print(vars(i)['en'])


['数え切れ', 'ない', '程', ' ', '長い', '時間', 'を']
['for', 'the', 'countless', ',', 'countless', 'hours']
['つまりね', ' ', 'この', '仕事', 'で', ' ', '大嫌い', 'な', 'こと', 'は', 'たく', 'さん', 'ある', 'わ']
['i', 'mean', ',', 'there', "'s", 'a', 'lot', 'of', 'things', 'about', 'this', 'job', 'that', 'i', 'hate']
['啊', ' ', '没什', '么']
['i', "'m", 'sorry']


In [68]:
def cal_bleu_score(dataset_pairs):
    targets = []
    predictions = []
 
    for i in dataset_pairs:
        target = vars(i)['jp']
        target = ' '.join(target)
        predicted_words = translate_sentence(vars(i)['en'], SRC, TRG, model, device)
        predictions.append(' '.join(predicted_words))
        targets.append(target)
    print(predictions[:3])
    print(targets[:3])
    print(f'BLEU Score: {round(corpus_bleu(predictions, targets) * 100, 2)}')

In [70]:
len(valid_data.examples)

80000

In [69]:
cal_bleu_score(valid_data.examples)

['<unk> の     <unk> の', '<unk> の <unk> の  ', '<unk> の                              ']
['♪   道選 べる', 'そこ で 私 は', '「表 に は 出 て こない 大物 と も コネ が ある の よ 」']
BLEU Score: 49.37


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


На всякий случай сохраню этот вариант, и попробую переобучить,если успею.